In [5]:
import Pyro5.api
import os
from glob import glob
import json

from imagedaemon.utils.paths import RAW_DATA_DIR
from imagedaemon.utils.wcs_utils import pix2sky


ns = Pyro5.api.locate_ns(host="localhost")

#winter = Pyro5.api.Proxy(ns.lookup("winter_daemon"))
#wcs = winter.get_astrometry(addr="pb", science_image="IMG.fits", ra=..., dec=...)

qcmos = Pyro5.api.Proxy(ns.lookup("qcmos_daemon"))

In [ ]:


image_dir = os.path.join(RAW_DATA_DIR, "qcmos", "science")
science_images = sorted(glob(os.path.join(image_dir, "*8s*.fits")))
science_image = science_images[0]
print(f"Science image: {science_image}")

# run the full astrometric pipeline
output_dir = os.path.join(os.path.join(os.getenv("HOME"), "data", "tmp"))

# this image has no RA/Dec in the header, so we need to set it
ra_guess = 83.6
dec_guess = 22



astrom_info = qcmos.solve_astrometry(science_image = science_image,
                                      background_image_list = None,
                                      output_dir = output_dir,
                                      ra = ra_guess,
                                      dec = dec_guess,
                                      pix_coords = None,
    )

print(f"Astrometric solution: {json.dumps(astrom_info, indent = 2)}")

# the image is 4096 x 2304
wx = 4096
wy = 2304
# let's try to get the WCS for a pixel in the center of the image
x_pix, y_pix = wx // 2, wy // 2
sky_pos = pix2sky(x_pix, y_pix, astrom_info["wcs"])
print(
    f"Coord conversion: (x, y) = ({x_pix}, {y_pix}) -> (RA, Dec) = ({sky_pos.ra:.1f}, {sky_pos.dec:.1f}) deg"
)
#best_focus = qcmos.run_focus_loop(image_list=[...])

Science image: /Users/nlourie/data/image-daemon-data/raw/qcmos/science/8s_image_single.cal.fits


TypeError: scalar 'SkyCoord' object is not iterable.